# Introduction : Modern Portfolio Theory

**Modern Portfolio Theory** est une théorie de l'investissement qui essaye de maximiser le rendement attendu sur la base d'un niveau de risque donné en choisissant minutieusement la proportions d'actifs dans le portefeuille.

Un portefeuille est un regroupement de différents types d'investissements tels que des actions, des obligations, des matières premières, des biens immobiliers, etc.

La théorie du portefeuille moderne suggère que le rendement attendu d'un investissement est directement lié au risque associé. Elle suggère qu'un investisseur doit choisir un portefeuille en fonction de son aversion au risque et de son horizon de placement.

Nous allons développer ici du code Python pour calculer les rendements attendus et la volatilité (risque) d'un portefeuille.

On utilise la bibliothèque pandas_datareader pour récupérer les données, numpy pour les opérations mathématiques, matplotlib pour visualiser et scipy pour l'optimisation.

Dans la suite de ce notebook, nous allons utiliser les notations suivantes :

- $R_p$ : rendement du portefeuille
- $w_i$ : proportion de l'actif $i$ dans le portefeuille
- $R_i$ : rendement de l'actif $i$
- $\Sigma_i$ : volatilité de l'actif $i$
- $\Sigma_p$ : volatilité du portefeuille
    - Les variables en majuscules sont des variables aléatoires
    - Les variables en minuscules sont deterministes
    - Les variables en gras sont des vecteurs.
----------

Notre objectif est de trouver les proportions $w_i$ qui maximisent le rendement du portefeuille pour un niveau de risque donné :

$$
\begin{align}
\max_{w_i} \quad & \mathbb{E}[R_p] \\
\text{sous contrainte} \quad & \mathbb{V}[R_p] < \sigma_{max}^2
\end{align}
$$

On définit le rendement du portefeuille comme la somme des rendements des actifs pondérés par leur proportion dans le portefeuille :

$$
R_p = \sum_i w_i R_i = \mathbf{w}^T \mathbf{R_p}
$$
Ainsi la variance du portefeuille est :

$$
\begin{align}
\mathbb{V}[R_p] &= \mathbb{V}[\mathbf{w}^T \mathbf{R_p}] \\
&= \mathbf{w}^T \mathbb{V}[\mathbf{R_p}] \mathbf{w} \\
&= \mathbf{w}^T \mathbf{\Sigma_p} \mathbf{w}
\end{align}
$$

Où $\mathbf{\Sigma_p}$ est la matrice de covariance des rendements des actifs du portefeuille. Pour rappel, la matrice de covariance est définie comme suit :

$$
\mathbf{\Sigma_p} = \begin{bmatrix}
\mathbb{V}[R_1] & \mathbb{C}[R_1, R_2] & \cdots & \mathbb{C}[R_1, R_n] \\
\mathbb{C}[R_2, R_1] & \mathbb{V}[R_2] & \cdots & \mathbb{C}[R_2, R_n] \\
\vdots & \vdots & \ddots & \vdots \\
\mathbb{C}[R_n, R_1] & \mathbb{C}[R_n, R_2] & \cdots & \mathbb{V}[R_n]
\end{bmatrix}
$$

Il nous reste donc à définir les $R_i$ et $\Sigma_i$ pour chaque actif du portefeuille de manière empirique. On notant $P_t$ le prix de l'actif $i$ à la date $t$, on définit le rendement de l'actif $i$ comme :

$$
R_i = \frac{P_t - P_{t-1}}{P_{t-1}} \text{ et } \Sigma_i = \sqrt{\mathbb{V}[R_i]}
$$



 

In [ ]:
#pip install numpy pandas matplotlib scipy pandas_datareader plotly nbformat

In [1]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader as web
import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
import datetime as dt
pio.templates.default = "plotly_dark"

## Etape 1 : Récupération des données

Dans cette étape, nous allons récupérer les données des actifs qui composent notre portfolio. Pour cet exemple, nous allons considérer 3 actions : Google ('GOOG'), Apple ('AAPL') et Microsoft ('MSFT'). Mais vous pouvez ajouter autant d'actions que vous le souhaitez.

In [2]:
def get_data(assets, start_date, end_date):
    """
    Récupère les données des actions sur Yahoo Finance
    """
    return yf.download(assets, start=start_date, end=end_date)['Adj Close']

In [3]:
# Liste des actifs dans le portefeuille
assets = ['AAPL', 'AMZN', 'MSFT', 'GOOG', 'TSLA', 'NVDA', 'ADBE', 'NFLX']
N = 365
start_date = dt.datetime.now() - dt.timedelta(days=N)
end_date = dt.datetime.now()

data = get_data(assets, start_date, end_date)

[*********************100%%**********************]  8 of 8 completed


In [4]:
# Affichage des données superposé avec plotly pour une année 
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=data.index, y=data[a], name=a))
fig.update_layout(title='Evolution des prix des actifs', yaxis_title='Prix en $')
fig.show()

Comme nous avons vu dans le cours, pour faire de l'optimisation de portefeuille, nous avons besoin des rendements historiques des actifs. Nous allons donc récupérer les données sur une période de 5 ans.

## Etape 2 : Calcul des returns

Dans cette étape, nous allons calculer les rendements quotidiens des actions. Les rendements sont calculés en utilisant la formule suivante :
$$
R_t = \frac{P_{t} - P_{t-1}}{P_{t-1}}
$$
Où $P_{t}$ est le prix à la date $t$ et $P_{t-1}$ est le prix à la date $t-1$.
En fait, nous calculons le pourcentage de changement de prix entre deux jours consécutifs, c'est pour cela que l'on divise par $P_{t-1}$.

In [5]:
def compute_portfolio_return(data):
    return data.pct_change()

In [6]:
returns = compute_portfolio_return(data)
# Affichage des rendements journaliers
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=returns.index, y=returns[a], name=a))
fig.update_layout(title='Rendements journaliers des actifs', yaxis_title='Rendements')
fig.show()

Ici nous avons accès aux réalisations des rendements $R_i(\omega)$ et nous voulons estimer les paramètres de la distribution des rendements ($\mu$ et $\sigma$). 

In [7]:
# Returns distribution
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Histogram(x=returns[a], name=a))
fig.update_layout(title='Distribution des rendements journaliers', yaxis_title='Fréquence')
fig.show()

Nous avons accès à $\mathbb{E}[R_i]$ et $\mathbb{V}[R_i]$. Stockons ces valeurs dans un dataframe.

In [8]:
# stock expected return and volatility in one dataframe
returns_mean = returns.mean()
returns_std = returns.std()
returns_mean_std = pd.concat([returns_mean, returns_std], axis=1)
returns_mean_std.columns = ['Mean', 'Std']
returns_mean_std

,Mean,Std
AAPL,0.001350,0.013626
ADBE,0.002565,0.019986
AMZN,0.002153,0.021292
GOOG,0.001680,0.019654
MSFT,0.001787,0.016223
NFLX,0.001808,0.024572
NVDA,0.004536,0.031508
TSLA,0.002009,0.036203


## Etape 3 : Matrice de covariance

Maintenant que nous avons les rendements historiques des actifs, nous pouvons calculer la matrice de covariance des rendements. La matrice de covariance est définie comme suit :

$$
\mathbf{\Sigma_p} = \begin{bmatrix}
\mathbb{V}[R_1] & \mathbb{C}[R_1, R_2] & \cdots & \mathbb{C}[R_1, R_n] \\
\mathbb{C}[R_2, R_1] & \mathbb{V}[R_2] & \cdots & \mathbb{C}[R_2, R_n] \\
\vdots & \vdots & \ddots & \vdots \\
\mathbb{C}[R_n, R_1] & \mathbb{C}[R_n, R_2] & \cdots & \mathbb{V}[R_n]
\end{bmatrix}
$$

La covariance empirique entre deux actifs $i$ et $j$ est définie comme suit :

$$
\mathbb{C}[R_i, R_j] = \frac{1}{T-1} \sum_{t=1}^T (R_i(t) - \mathbb{E}[R_i])(R_j(t) - \mathbb{E}[R_j])
$$


In [9]:
# Calcul de la matrice de correlation
corr_matrix = returns.corr()
cov_matrix = returns.cov()
# Affichage de la matrice de correlation with x axis and y axis with the same size
fig = go.Figure(data=go.Heatmap(z=corr_matrix, x=assets, y=assets, colorscale='Darkmint'))
fig.update_layout(
    margin=dict(l=350, r=350, t=50, b=50),
)
fig.update_layout(title='Matrice de corrélation', yaxis_title='Actifs', xaxis_title='Actifs')
fig.show()



On peut maintenant essayer de visualiser calculer la valeur de notre portefeuille pour différentes valeurs de $w_i$.

In [11]:
# On va donc créer une fonction qui prend en entrée un vecteur de poids et qui retourne la valeur du portefeuille.
def compute_portfolio_value(weights, returns):
    """
    weights: array of weights
    returns: dataframe of returns
    """
    return np.dot(returns.mean(), weights) * len(returns)

def compute_portfolio_risk(weights, returns):
    """
    weights: array of weights
    returns: dataframe of risk
    """
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights))) * np.sqrt(len(returns))

w_test = np.array([1/len(assets)]*len(assets))
portfolio_return = compute_portfolio_value(w_test, returns)

initial_portfolio_value = 100

print('Valeur du portefeuille: ', np.round((1+portfolio_return) * (initial_portfolio_value)), '$')
print('Risque du portefeuille: ', np.round(compute_portfolio_risk(np.array([1/len(assets)]*len(assets)), returns), 4))

Valeur du portefeuille:  156.0 $
Risque du portefeuille:  0.2599


# Etape 4 : Optimisation du portefeuille

Maintenant que nous avons toutes les informations nécessaires, nous pouvons commencer à optimiser notre portefeuille. 

$$
\begin{align}
\max_{w_i} \quad & \mathbf{w}^T \mathbf{R_p} \\
\text{sous contrainte} \quad & \mathbf{w}^T \mathbf{\Sigma_p} \mathbf{w} < \sigma_{max}^2
\end{align}
$$

Utilisons Monte Carlo pour générer des valeurs aléatoires de $w_i$ et calculons le rendement et la volatilité du portefeuille pour chaque valeur de $w_i$.

In [45]:

def generate_random_weights(n):
    """
    n: number of weights to generate with dirichlet distribution
    """
    weights = np.random.dirichlet(np.ones(n), size=1)
    return weights[0]

def generate_random_portfolio(returns):
    """
    returns: dataframe of returns
    """
    weights = generate_random_weights(len(returns.columns))
    
    portfolio_return = compute_portfolio_value(weights, returns)
    portfolio_risk = compute_portfolio_risk(weights, returns)

    return weights, portfolio_return, portfolio_risk

def generate_random_portfolios(returns, n):
    """
    returns: dataframe of returns
    n: number of portfolios to generate
    """
    portfolios = []
    for i in range(n):
        weights, portfolio_return, portfolio_risk = generate_random_portfolio(returns)
        portfolios.append([weights, portfolio_return, portfolio_risk])
    portfolios_df = pd.DataFrame(portfolios, columns=['weights', 'returns', 'risks'])
    return portfolios_df


In [46]:

# On va générer 10000 portefeuilles aléatoires et les afficher sur un graphique.
N = 10000
portfolios_df = generate_random_portfolios(returns, N)

### Visualisation des résultats

In [47]:
fig = go.Figure()
fig.update_layout(
    margin=dict(l=200, r=200, t=50, b=50),
)
fig.add_trace(go.Scatter(x=portfolios_df['risks'], y=portfolios_df['returns'], mode='markers', marker=dict(color=portfolios_df['returns']/portfolios_df['risks'], showscale=True, colorscale='RdBu', size=7)))

# On va ajouter une etoile pour le portefeuille avec le meilleur sharpe ratio
best_sharpe_ratio_index = (portfolios_df['returns']/portfolios_df['risks']).idxmax()
fig.add_trace(go.Scatter(x=[portfolios_df['risks'][best_sharpe_ratio_index]], y=[portfolios_df['returns'][best_sharpe_ratio_index]], mode='markers', marker=dict(color='yellow', size=15, symbol='star')))

# On va ajouter une croix pour le portefeuille avec le risque le plus faible
lowest_risk_index = portfolios_df['risks'].idxmin()
fig.add_trace(go.Scatter(x=[portfolios_df['risks'][lowest_risk_index]], y=[portfolios_df['returns'][lowest_risk_index]], mode='markers', marker=dict(color='yellow', size=15, symbol='x')))


fig.update_layout(title='Frontière efficiente', yaxis_title='Rendements', xaxis_title='Risques')
fig.show()

Maintenant que nous avons calculé les rendements et la volatilité du portefeuille pour différentes valeurs de $w_i$, nous pouvons choisir la valeur de $w_i$ qui maximise le rendement du portefeuille pour un niveau de risque donné.

In [48]:
SIGMA_MAX = 0.3

# le poids qui maximise le rendement pour un risque donné inférieur à SIGMA_MAX
best_weights_index = portfolios_df[portfolios_df['risks'] < SIGMA_MAX]['returns'].idxmax()
best_weights = portfolios_df['weights'][best_weights_index]

best_portfolio_return = compute_portfolio_value(best_weights, returns)
best_portfolio_risk = compute_portfolio_risk(best_weights, returns)

initial_portfolio_value = 100

print('Valeur du portefeuille: ', np.round((1+best_portfolio_return) * (initial_portfolio_value)), '$')

fig = go.Figure()
fig.update_layout(
    margin=dict(l=200, r=200, t=50, b=50),
)
fig.add_trace(go.Bar(x=assets, y=best_weights))
fig.update_layout(title='Poids des actifs', yaxis_title='Poids', xaxis_title='Actifs')
fig.show()

Valeur du portefeuille:  173.0 $


### Calcul de la frontière efficiente

In [24]:

def calculate_frontier(returns):
    """
    returns: dataframe of returns
    """
    N = len(returns.columns)
    initial_weights = np.array([1/N]*N)
    bounds = tuple((0, 1) for asset in range(N))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    frontier_risks = []
    frontier_returns = []
    frontier_weights = []
    lowest_risk_index = portfolios_df['risks'].idxmin()
    min_return = portfolios_df['returns'][lowest_risk_index]
    for portfolio_return in np.linspace(min_return, 1, 100):
        constraints = ({'type': 'eq', 'fun': lambda x: compute_portfolio_value(x, returns) - portfolio_return},
                       {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        efficient_portfolio = minimize(compute_portfolio_risk, initial_weights, args=returns, method='SLSQP', bounds=bounds, constraints=constraints)
        frontier_risks.append(efficient_portfolio.fun)
        frontier_returns.append(portfolio_return)
        frontier_weights = efficient_portfolio.x
    return frontier_risks, frontier_returns, frontier_weights

frontier_risks, frontier_returns, w = calculate_frontier(returns)


In [29]:
fig = go.Figure()
fig.update_layout(
    margin=dict(l=200, r=200, t=50, b=50),
)
fig.add_trace(go.Scatter(x=portfolios_df['risks'], y=portfolios_df['returns'], mode='markers', marker=dict(color=portfolios_df['returns']/portfolios_df['risks'], showscale=True, colorscale='RdBu', size=7)))
fig.add_trace(go.Scatter(x=frontier_risks, y=frontier_returns, mode='lines', line=dict(color='yellow', width=3), name='Frontière efficiente'))
fig.update_layout(title='Frontière efficiente', yaxis_title='Rendements', xaxis_title='Risques')
fig.show()

Etant donné $\sigma_{max}$, nous pouvons calculer le rendement maximum du portefeuille à l'aide de la frontière efficiente.

# Etape 5 : Test d'une stratégie d'investissement

Définissons une stratégie d'investissement simple :

- Tout les jours, nous allons rééquilibrer notre portefeuille en choisissant les proportions $w_i$ qui maximisent le rendement du portefeuille pour un niveau de risque donné. En regardant les données des $N$ jours précédents.
- Nous allons investir initialement $S$ dans notre portefeuille.

Nous allons maintenant tester notre stratégie d'investissement sur une période de 5 ans.

Ici, $S = 1000$, $N = 30$ et $\sigma_{max} = 0.3$.

### Simulation de la stratégie d'investissement

In [89]:
S = 1000
N = 10
SIGMA_MAX = 0.4
number_of_days_to_test = 50
portfolio_value_history = []
portfolio_value_history_without_rebalancing = []
s = S
portfolio_weights_history = []

first_date = dt.datetime.now() - dt.timedelta(days=730+100)
current_date = dt.datetime.now() - dt.timedelta(days=730)
total_data = get_data(assets, first_date,  dt.datetime.now())

for i in range(number_of_days_to_test):
    print('Simulation: ', i)
    # On récupère les données des N derniers jours
    current_data = total_data[first_date:current_date]

    next_data = total_data[current_date:current_date + dt.timedelta(days=N)]
    # On calcul les rendements
    past_returns = compute_portfolio_return(current_data)
    next_returns = compute_portfolio_return(next_data)

    portfolios_df = generate_random_portfolios(past_returns, 10000)

    # le poids qui maximise le rendement pour un risque donné inférieur à SIGMA_MAX
    # check if there is a portfolio with a risk lower than SIGMA_MAX
    if not portfolios_df[portfolios_df['risks'] < SIGMA_MAX].empty:
        best_weights_index = portfolios_df[portfolios_df['risks'] < SIGMA_MAX]['returns'].idxmax()
        best_weights = portfolios_df['weights'][best_weights_index]
    else:
        best_weights = portfolio_weights_history[-1]

    # On calcul le poids du portefeuille
    portfolio_return = compute_portfolio_value(best_weights, next_returns)
    portfolio_return_without_rebalancing = compute_portfolio_value(np.array([1/len(assets)]*len(assets)), next_returns)

    S = (1+portfolio_return) * S
    s = (1+portfolio_return_without_rebalancing) * s

    portfolio_value_history.append(S)
    portfolio_value_history_without_rebalancing.append(s)
    portfolio_weights_history.append(best_weights)

    current_date += dt.timedelta(days=N)


[*********************100%%**********************]  8 of 8 completed
Simulation:  0
Simulation:  1
Simulation:  2
Simulation:  3
Simulation:  4
Simulation:  5
Simulation:  6
Simulation:  7
Simulation:  8
Simulation:  9
Simulation:  10
Simulation:  11
Simulation:  12
Simulation:  13
Simulation:  14
Simulation:  15
Simulation:  16
Simulation:  17
Simulation:  18
Simulation:  19
Simulation:  20
Simulation:  21
Simulation:  22
Simulation:  23
Simulation:  24
Simulation:  25
Simulation:  26
Simulation:  27
Simulation:  28
Simulation:  29
Simulation:  30
Simulation:  31
Simulation:  32
Simulation:  33
Simulation:  34
Simulation:  35
Simulation:  36
Simulation:  37
Simulation:  38
Simulation:  39
Simulation:  40
Simulation:  41
Simulation:  42
Simulation:  43
Simulation:  44
Simulation:  45
Simulation:  46
Simulation:  47
Simulation:  48
Simulation:  49


### Affichage de l'évolution de la valeur du portefeuille

In [90]:
# plot portfolio value history and portfolio value history without rebalancing
fig = go.Figure()
fig.update_layout(
    margin=dict(l=200, r=200, t=50, b=50),
)
fig.add_trace(go.Scatter(x=[first_date + dt.timedelta(i*N) for i in range(len(portfolio_value_history_without_rebalancing))], y=portfolio_value_history, mode='lines', line=dict(color='yellow', width=3), name='Avec rebalancement'))
fig.add_trace(go.Scatter(x=[first_date + dt.timedelta(i*N) for i in range(len(portfolio_value_history_without_rebalancing))], y=portfolio_value_history_without_rebalancing, mode='lines', line=dict(color='red', width=3), name='Sans rebalancement'))
fig.update_layout(title='Evolution du portefeuille', yaxis_title='Valeur du portefeuille', xaxis_title='Jours')
fig.show()

### Affichage de l'évolution des poids des actifs dans le portefeuille

In [91]:
weights_df = pd.DataFrame(portfolio_weights_history, columns=[assets[i] for i in range(len(assets))])

cumulative_weights = weights_df.cumsum(axis=1)

fig = go.Figure()
for column in cumulative_weights.columns:
    fig.add_trace(go.Scatter(
        x=[first_date + dt.timedelta(i*N) for i in cumulative_weights.index],
        y=cumulative_weights[column],
        fill='tonexty',
        mode='lines',
        name=column
    ))

fig.update_layout(
    title='Évolution cumulée des poids des actifs',
    yaxis_title='Poids cumulés',
    xaxis_title='Date'
)
fig.show()